In [5]:
!pip install speechbrain

In [13]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
from scipy import spatial


In [31]:
class SpeakerVerifier():
  
  def __init__(self):
    self.classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", 
                                                savedir="pretrained_models/spkrec-xvect-voxceleb")
    self.enrolled_embeddings = []
    self.version = "0.0.1"
    self.enrolled_speaker = []


  def enroll(self, user_voice_path, speaker_name):
    signal, fs = torchaudio.load(user_voice_path)
    embedding = self.classifier.encode_batch(signal)
    self.enrolled_embeddings.append(embedding[0][0].numpy())
    self.enrolled_speaker.append(speaker_name)


  def verity(self, user_voice_path,name):
    signal, fs = torchaudio.load(user_voice_path)
    embedding = self.classifier.encode_batch(signal)[0][0].numpy()

    cosine_min = 100000
    for i,speaker in enumerate(self.enrolled_embeddings):
      cosine_distance = spatial.distance.cosine(embedding, speaker)

      if cosine_distance < cosine_min:
        cosine_min = cosine_distance
        detected_speaker = self.enrolled_speaker[i]

    return name == detected_speaker  
      
      

In [32]:
speaker_verifier = SpeakerVerifier()


In [33]:
speaker_verifier.enroll("maryam_sv.wav", "maryam")

In [35]:
speaker_verifier.enroll("mohammad_sv.wav", "mohammad")

In [37]:
speaker_verifier.verity("maryam_sv.wav", "maryam")

True

In [39]:
speaker_verifier.verity("maryam_sv.wav", "mohammad")

False

In [40]:
speaker_verifier.verity("mohammad_sv.wav", "maryam")

False

In [41]:
speaker_verifier.verity("mohammad_sv.wav", "mohammad")

True